# [SQL Generation in Text2SQL with TinyLlama's LLM Fine-tuning (analyticsvidhya.com)](https://www.analyticsvidhya.com/blog/2024/02/sql-generation-in-text2sql-with-tinyllamas-llm-fine-tuning/)

In [ ]:
!nvidia-smi

Thu Apr 11 05:01:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:00:05.0 Off |                    0 |
| N/A   40C    P0              45W / 300W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## Setting Up the Environment

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install llama-cpp-python
!pip3 install huggingface-hub
!pip3 install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 23.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.61-cp310-cp310-manylinux_2_35_x86_64.whl size=39013369 sha256=702c1cfce710b14bc69e25aa1689ccf87ff267c238e08ef4a15a17f62d26a1b8
  Stored in directory: /root/.cache/pip/wheels/a4/d4/bc/483ff5c198ee745418fb35b59055591eeae2bc6ff69680537f
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pi

- CMAKE_ARGS="-DLLAMA_CUBLAS=on": llama-cpp-python을 빌드하는 동안 CUBLAS 라이브러리를 사용하여 GPU 가속을 활성화합니다.
- FORCE_CMAKE=1: cmake를 강제로 실행하여 새로운 빌드를 보장합니다.
- pip3 install llama-cpp-python: 양자화된 모델과 상호작용하려면  라이브러리가 필요합니다.
- !pip3 install accelerate peft bitsandbytes transformers trl:
```
accelerate: 훈련 프로세스를 여러 GPU 또는 machine에 분산시켜 훈련 시간을 크게 단축시킬 수 있음

peft: 사용자 정의 데이터 세트에서 대규모 언어 모델을 미세 조정하기 위한 도구 및 기술 제공

Bitsandbytes: LLM의 메모리 사용량을 줄여 메모리 리소스가 제한된 시스템에서 교육할 수 있도록 도와줌

trl: 의사 결정 및 계획이 필요한 작업을 위해 LMM을 미세 조정하는 데 사용할 수 있는 강화 학습용 알고리즘 및 도구 제공
```

## Modeling

**model downlaod**

In [ ]:
from huggingface_hub import hf_hub_download

# model_name: HuggingFace 모델 허브의 모델 이름
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"

# Define the name of the model file to download.
model_file = "tinyllama-1.1b-chat-v1.0.Q8_0.gguf"

# Download the model from the Hugging Face Hub and store the
# path to the downloaded file in the `model_path` variable.
# model_path: HuggingFace에서 모델을 다운로드 한 후 모델 경로가 저장될 변수
model_path = hf_hub_download(model_name, filename=model_file)

# Print a message indicating that the model has been downloaded.
print(f"Model downloaded to: {model_path}")

tinyllama-1.1b-chat-v1.0.Q8_0.gguf:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q8_0.gguf


**Initializing the Model**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from llama_cpp import Llama

# Initialize a `Llama` object with the downloaded model path.
# Llama: Class from llama_cpp library, that is worked with to initialize the model.
llm = Llama(
    # model_path: It is the path to the downloaded model that we obtained a while ago
    model_path=model_path,

    # Set the number of context tokens.
    # n_ctx: This variable takes in the number of context tokens the model can handle. Here we are passing it a value of 512
    n_ctx=512,

    # Set the number of threads to use.
    # n_threads: This variable takes in the number of CPU threads for computation. The Google Colab has a 4-core CPU, hence passing it 8 threads
    n_threads=8,

    # Set the number of GPU layers to work with.
    # n_gpu_layers: This variable takes in the number of GPU layers in which the model needs to be offloaded. The value of 40 will offload the entire TinyLlama 1.1B within the Google Colab T4 GPU
    n_gpu_layers=40
)

# Print a message indicating that the Llama object has been initialized.
print("Llama object initialized successfully.")

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /root/.cache/huggingface/hub/models--TheBloke--TinyLlama-1.1B-Chat-v1.0-GGUF/snapshots/52e7645ba7c309695bec7ac98f4f005b139cf465/tinyllama-1.1b-chat-v1.0.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader:

Llama object initialized successfully.


In [ ]:
# Use the Llama object to generate an answer to the question.
output = llm(
    # Prompt
    "<|im_start|>user\nAre you a robot?<|im_end|>\n<|im_start|>assistant\n",

    # Set the maximum number of tokens to generate.
    max_tokens=512,

    # Set the stop sequences to indicate the end of the generated text.
    stop=["</s>"],
)

# Print the generated text.
print(output['choices'][0]['text'])


llama_print_timings:        load time =    8652.62 ms
llama_print_timings:      sample time =      47.58 ms /   297 runs   (    0.16 ms per token,  6242.51 tokens per second)
llama_print_timings: prompt eval time =    8652.48 ms /    33 tokens (  262.20 ms per token,     3.81 tokens per second)
llama_print_timings:        eval time =    1259.11 ms /   296 runs   (    4.25 ms per token,   235.09 tokens per second)
llama_print_timings:       total time =   10531.77 ms /   329 tokens


I am not a robot. However, I can provide some general information about robots. Robots are machines designed to perform specific tasks and functions without human intervention. They may be programmed to perform repetitive or routine tasks, such as assembly lines or manufacturing plants, or they may be autonomous and self-sufficient. The primary purpose of a robot is to perform a task in an efficient and cost-effective manner while minimizing risks or errors associated with human intervention. Some common types of robots include automated machinery, such as assembly lines and manufacturing equipment; mobile robotics (such as self-driving cars); humanoid robots (such as the humanoid robot that Apollo 11 astronauts used for the moon landing); and humanoid robotic systems (such as the AI system in Star Trek). The use of robots in industry and science has grown rapidly over the past few decades, with advancements in technology making them more efficient, cost-effective, and reliable. Some c

## Testing the Vanilla TinyLlama

Context와 Question을 제공하면 SQL 쿼리를 만드는 함수

In [ ]:
def chat_template(question, context):
    """
    Creates a chat template for the Llama model.

    Args:
        question: The question to be answered.
        context: The context information to be used for generating the answer.

    Returns:
        A string containing the chat template.
    """

    template = f"""\
    <|im_start|>user
    Given the context, generate an SQL query for the following question
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

**input으로 들어가게 될 template**

In [ ]:
question = "How many heads of the departments are older than 56 ?"
context = "CREATE TABLE head (age INTEGER)"
print(chat_template(question,context))

<|im_start|>user
Given the context, generate an SQL query for the following question
context:CREATE TABLE head (age INTEGER)
question:How many heads of the departments are older than 56 ?
<|im_end|>
<|im_start|>assistant 



**output 출력**

In [ ]:
# Use the Llama object to generate an answer to the question.
output = llm(
    chat_template(question, context),

    # Set the maximum number of tokens to generate.
    max_tokens=512,

    # Set the stop sequences to indicate the end of the generated text.
    stop=["</s>"],
)


# Print the generated text.
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8652.62 ms
llama_print_timings:      sample time =      11.12 ms /    72 runs   (    0.15 ms per token,  6475.98 tokens per second)
llama_print_timings: prompt eval time =       9.31 ms /    62 tokens (    0.15 ms per token,  6658.08 tokens per second)
llama_print_timings:        eval time =     299.74 ms /    71 runs   (    4.22 ms per token,   236.87 tokens per second)
llama_print_timings:       total time =     447.52 ms /   133 tokens


To find the number of heads of the Department in which age is greater than or equal to 56, we can use a query like this:
```sql
SELECT COUNT(*) FROM head WHERE age >= 56;
```
This will count all heads in the `head` table where `age` is greater than 56.
